In [21]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor

# Cargar datos
costos_df = pd.read_excel('Costos_Subprocesos.xlsx')
indicadores_df = pd.read_excel('Indicadores_Minas.xlsx')

# Convertir las fechas a formato datetime
costos_df['Subproceso_Costo'] = pd.to_datetime(costos_df['Subproceso_Costo'])
indicadores_df['Fecha'] = pd.to_datetime(indicadores_df['Fecha'])

# Unir los DataFrames por la columna de fechas
df_unificado = pd.merge(costos_df, indicadores_df, left_on='Subproceso_Costo', right_on='Fecha', how='inner')
df_unificado = df_unificado.drop(columns=['Fecha'])

# Calcular la correlación entre todos los subprocesos y los indicadores
subprocesos_cols = costos_df.columns[1:]  # Excluyendo la columna de fecha
indicadores_cols = indicadores_df.columns[1:]  # Excluyendo la columna de fecha
correlation_matrix = df_unificado.corr()
correlation_filtered = correlation_matrix.loc[subprocesos_cols, indicadores_cols]
top_correlations = correlation_filtered.apply(lambda x: x.nlargest(3), axis=1)

# Dividir los datos en entrenamiento (2016-2021) y prueba (2022)
train_data = df_unificado[df_unificado['Subproceso_Costo'] < '2022-01-01']
test_data = df_unificado[(df_unificado['Subproceso_Costo'] >= '2022-01-01') & (df_unificado['Subproceso_Costo'] < '2023-01-01')]

# Definir una lista de modelos a evaluar
modelos = {
    'Linear Regression': LinearRegression(),
    'Polynomial Regression': LinearRegression(),  # Usaremos PolynomialFeatures más adelante
    'Neural Network': MLPRegressor(hidden_layer_sizes=(100,), max_iter=1000, random_state=42),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
    'XGBoost': XGBRegressor(n_estimators=100, random_state=42)
}

# Crear un diccionario para almacenar los resultados de cada modelo
resultados_modelos = {}

# Realizar predicciones para cada modelo
for nombre_modelo, modelo in modelos.items():
    resultados = {}
    for subproceso in subprocesos_cols:
        indicadores_seleccionados = top_correlations.loc[subproceso].dropna().index.tolist()
        if len(indicadores_seleccionados) > 0:
            X_train = train_data[indicadores_seleccionados]
            y_train = train_data[subproceso]
            X_test = test_data[indicadores_seleccionados]
            
            if nombre_modelo == 'Polynomial Regression':
                poly = PolynomialFeatures(degree=2)
                X_train = poly.fit_transform(X_train)
                X_test = poly.transform(X_test)
                modelo.fit(X_train, y_train)
            else:
                modelo.fit(X_train, y_train)
            
            y_pred_2022 = modelo.predict(X_test)
            costo_real_2022 = costos_df[costos_df['Subproceso_Costo'] >= '2022-01-01'][subproceso].values
            
            # Calcular las diferencias y diferencias porcentuales
            diferencias = y_pred_2022 - costo_real_2022
            diferencias_pct = [(dif / real) * 100 if real != 0 else 'N/A' for dif, real in zip(diferencias, costo_real_2022)]
            
            # Guardar resultados en el diccionario
            for mes, pred, real, dif, dif_pct in zip(pd.date_range('2022-01-01', '2022-12-01', freq='MS').strftime("%B-%Y"), y_pred_2022, costo_real_2022, diferencias, diferencias_pct):
                resultados.setdefault(subproceso, {}).update({
                    f'Predicción {mes}': pred,
                    f'Costo Real {mes}': real,
                    f'Diferencia {mes}': dif,
                    f'Diferencia % {mes}': dif_pct
                })
    
    # Convertir los resultados a un DataFrame
    resultados_df = pd.DataFrame(resultados).T
    resultados_modelos[nombre_modelo] = resultados_df

# Crear DataFrames para almacenar los mejores modelos y sus diferencias
mejores_modelos_df = pd.DataFrame(index=subprocesos_cols, columns=pd.date_range('2022-01-01', '2022-12-01', freq='MS').strftime("%B-%Y"))
diferencias_df = pd.DataFrame(index=subprocesos_cols, columns=pd.date_range('2022-01-01', '2022-12-01', freq='MS').strftime("%B-%Y"))

# Comparar los resultados entre modelos para cada subproceso y mes
for subproceso in subprocesos_cols:
    for mes in pd.date_range('2022-01-01', '2022-12-01', freq='MS').strftime("%B-%Y"):
        mejor_modelo = None
        menor_diferencia = np.inf
        
        for nombre_modelo, df in resultados_modelos.items():
            diferencia_col = f'Diferencia % {mes}'
            if diferencia_col in df.columns and subproceso in df.index and df.loc[subproceso, diferencia_col] != 'N/A':
                diferencia = abs(df.loc[subproceso, diferencia_col])
                if diferencia < menor_diferencia:
                    menor_diferencia = diferencia
                    mejor_modelo = nombre_modelo
        
        # Guardar el mejor modelo y su diferencia
        mejores_modelos_df.loc[subproceso, mes] = mejor_modelo
        diferencias_df.loc[subproceso, mes] = menor_diferencia

# Exportar los resultados a un archivo Excel con dos hojas
output_file_path = r'C:\Users\Administrator\Documents\GitHub\Hackathon2-2024\1. Desarollo\Resultados\Costos_Indicadores_Mina_Mejor_Modelo_Final.xlsx'

with pd.ExcelWriter(output_file_path) as writer:
    mejores_modelos_df.to_excel(writer, sheet_name='Mejores Modelos')
    diferencias_df.to_excel(writer, sheet_name='Diferencias %')

# Mostrar los primeros registros de cada DataFrame
mejores_modelos_df.head(), diferencias_df.head()


C:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum it

(                                January-2022      February-2022  \
 ADM OPERACIÓN Y GESTIÓN ESP             None  Linear Regression   
 ADM OPERACIÓN Y GESTIÓN ESP SUR      XGBoost      Random Forest   
 ADM OPERACIÓN Y GESTIÓN LLANO           None               None   
 ADM OPERACIÓN Y GESTIÓN MIR             None     Neural Network   
 ADM OPERACIÓN Y GESTIÓN OXE             None            XGBoost   
 
                                             March-2022         April-2022  \
 ADM OPERACIÓN Y GESTIÓN ESP          Gradient Boosting      Random Forest   
 ADM OPERACIÓN Y GESTIÓN ESP SUR  Polynomial Regression  Linear Regression   
 ADM OPERACIÓN Y GESTIÓN LLANO                     None               None   
 ADM OPERACIÓN Y GESTIÓN MIR             Neural Network      Random Forest   
 ADM OPERACIÓN Y GESTIÓN OXE             Neural Network            XGBoost   
 
                                           May-2022       June-2022  \
 ADM OPERACIÓN Y GESTIÓN ESP      Gradient Boosti